In [1]:
list_patches = [
    "7.20", "7.20b", "7.20c", "7.20d", "7.20e", 
    "7.21", "7.21b", "7.21c", "7.21d", 
    "7.22", "7.22b", "7.22c", "7.22d", "7.22e", "7.22f", "7.22g", "7.22h",
    "7.23", "7.23a", "7.23b", "7.23c", "7.23d", "7.23e", "7.23f", 
    "7.24", "7.24b", 
    "7.25", "7.25a", "7.25b", "7.25c", 
    "7.26", "7.26a", "7.26b", "7.26c", 
    "7.27", "7.27a", "7.27b", "7.27c", "7.27d", 
    "7.28", "7.28a", "7.28b", "7.28c", 
    "7.29", "7.29b", "7.29c", "7.29d", 
    "7.30", "7.30b", "7.30c", "7.30d", "7.30e", 
    "7.31", "7.31b", "7.31c", "7.31d",
    "7.32", "7.32b", "7.32c", "7.32d", "7.32e",
    "7.33", "7.33b", "7.33c", "7.33d", "7.33e",
    "7.34", "7.34b", "7.34c", "7.34d", "7.34e",
    "7.35", "7.35b", "7.35c", "7.35d", 
    "7.36", "7.36a", "7.36b", "7.36c", 
    "7.37", "7.37b", "7.37c", "7.37d"
]

In [76]:
# import requests

# for p in list_patches:
# # URL of the Dota2 API endpoint
#     url = f'https://www.dota2.com/datafeed/patchnotes?version={p}&language=english'
#     response = requests.get(url)
#     data = response.json()
#     # Save into a file
#     # Save the JSON response to a file for JSONLoader to read
#     with open(f"patchnotes/{p}.json", "w") as f:
#         import json
#         json.dump(data, f)

In [23]:
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path="patchnotes/7.35.json",
    jq_schema='{.items[].ability_notes[].note}',
    text_content=True,
)

docs = loader.load()
docs

ValueError: jq: error: syntax error, unexpected FIELD (Windows cmd shell quoting issues?) at <top-level>, line 1:
{.items[].ability_notes[].note} 
jq: 1 compile error

In [83]:
from langchain_community.document_loaders import JSONLoader

docs = []

# Heroes
def extract_metadata(record: dict, metadata: dict) -> dict:
    metadata["hero_id"] = record.get("hero_id")
    metadata["abilities"] = record.get("abilities")
    metadata["category"] = "heroes"
    return metadata

loader_heroes = JSONLoader(
    file_path="patchnotes/7.37d.json",
    jq_schema=".heroes[]",
    text_content=False,
    metadata_func=extract_metadata,

)
docs_heroes = loader_heroes.load()
docs.append(docs_heroes)

# Items
def extract_metadata(record: dict, metadata: dict) -> dict:
    metadata["ability_id"] = record.get("ability_id")
    metadata["ability_notes"] = record.get("ability_notes")
    metadata["category"] = "items"
    return metadata

loader_items = JSONLoader(
    file_path="patchnotes/7.35c.json",
    jq_schema=".items[]",
    text_content=False,
    metadata_func=extract_metadata,
)
docs_items = loader_items.load()
docs.append(docs_items)

In [81]:
docs

[[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\7.37d.json', 'seq_num': 1, 'hero_id': 1, 'abilities': None, 'category': 'heroes'}, page_content='{"hero_id": 1, "subsections": [{"title": "Magebane\'s Mirror", "style": "hero_facet", "facet": "antimage_magebanes_mirror", "facet_icon": "ricochet", "facet_color": "Purple1", "abilities": [{"ability_id": 7314, "ability_notes": [{"indent_level": 1, "note": "Reflected Spell Amp increased from 30% to 40%"}]}, {"ability_id": 1143, "ability_notes": [{"indent_level": 1, "note": "Reflected Spell Amp increased from 30% to 40%"}]}]}], "talent_notes": [{"indent_level": 1, "note": "Level 10 Talent +10% Counterspell Magic Resistance replaced with +3 Health Regen"}]}'),
  Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\7.37d.json', 'seq_num': 2, 'hero_id': 4, 'abilities': [{'ability_id': 5015, 'ability_notes': [{'indent_level': 1, 'note': 

In [78]:
from typing import Sequence

import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [90]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

AttributeError: 'list' object has no attribute 'page_content'

In [25]:
### Construct retriever ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [26]:
import pprint
pprint.pprint(docs)


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [91]:
### Construct retriever ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [94]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [92]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()


In [5]:



### Construct retriever ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [6]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Common ways of task decomposition include using simple prompting techniques, such as asking for "Steps for XYZ" or "What are the subgoals for achieving XYZ?" Additionally, task-specific instructions can be employed, like "Write a story outline" for writing tasks, or human inputs can guide the decomposition process.
